# 1-Importing the Libraries

In [22]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


# 2-Importing the data

In [23]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'xml')
neightab=soup.find('table')

In [24]:
column = ['Postalcode','Borough','Neighborhood']
df=pd.DataFrame(columns=column)

In [25]:
for tr in neightab.find_all('tr'):
    tab_row=[]
    for td in tr.find_all('td'):
        row =td.text.strip()
        tab_row.append(row)
    if len(tab_row)==3:
        df.loc[len(df)] = tab_row

In [26]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# 3-Transforming the Data 

In [34]:
# Dropping the Not assigned'´s Borough rows
df1 = df[df.Borough != 'Not assigned']

#Grouping Neighbors with the same postal code
df2 =df1.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df2.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [35]:
# Assigning Borough to not assigned Neighbors
df2['Neighborhood']=np.where(df2['Neighborhood']=='Not assigned', df2['Borough'], df2['Neighborhood'])

In [36]:
df2

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [29]:
df2.shape

(103, 3)

# 4-Importing the Latitud and Longitud Data

In [12]:
cordlatlong = pd.read_csv('https://cocl.us/Geospatial_data')
cordlatlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# 5-Merging the Data

In [30]:
cordlatlong=cordlatlong.rename(columns={'Postal Code':'Postalcode'})
df4 = pd.merge(df2,cordlatlong, on = 'Postalcode')
df4.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# 6-Clustering the Data

In [37]:
df3 = merged_df[df4['Borough'].str.contains('Toronto',regex=False)]


#Run K-means with 4 clusters
nclusters = 4

dfcluster = df3.drop(['Postalcode','Borough','Neighborhood'], 1)


kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(dfcluster)

# check cluster labels generated 
kmeans.labels_[0:25] 

array([1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       2, 2, 3])

In [38]:
df3.insert(0, 'Cluster Number', kmeans.labels_)
df3.head()

,Cluster Number,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,1,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,1,M4M,East Toronto,Studio District,43.659526,-79.340923
44,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


# 7- Plotting the Clusters in the Map

In [39]:
clustermap = folium.Map(location=[43.6532,-79.3832],zoom_start=11)

# set color scheme for the clusters
x = np.arange(nclusters)
ys = [i + x + (i*x)**2 for i in range(nclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat,lng,borough,neighborhood,cluster in zip(df3['Latitude'],df3['Longitude'],df3['Borough'],df3['Neighborhood'],df3['Cluster Number']):
    #label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup('Toronto Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.75,
    parse_html=False).add_to(clustermap)
clustermap



## We finished segmenting the Toronto Area in 4 Clusters!